In [21]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
from keras import optimizers
from keras.layers import TimeDistributed
from time import time
from keras.callbacks import TensorBoard

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [22]:
from keras.models import load_model
model = load_model('avsr_kannada_testsame_v2.h5')

In [23]:
words_list = ['Avanu','Bagge','Bari', 'Howdu','Illa', 'Janarige', 'Kathe', 'Nale']
import os
import numpy as np
import pandas as pd
import imutils
from imutils import face_utils
import dlib
import cv2
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

SHAPE_PREDICTOR_LOCATION = "./shape_predictor_68_face_landmarks.dat"
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(SHAPE_PREDICTOR_LOCATION)

X  = []
Y  = []
label_dictionary = dict()


def process_Video(video):
    print("================================")
    X = []
    Y = []
    cap = cv2.VideoCapture(video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("FPS : " + str(fps))
    if(not cap.isOpened()):
        print(" .....Cannot open video stream or file!.....")
    print(" video/camera opened...")
    print("********************************************")
    while(cap.isOpened()):
        frameId = cap.get(1)
        try:
            ret, image = cap.read()
            if(ret==False):
                break
            else:
                print("Frame ID :"+ str(frameId))
                #image = imutils.rotate_bound(image,angle=90)
                #image = imutils.resize(image,width = 500)
                gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                #cv2.imshow("gray",image)
                faces = face_detector(gray_image,1)
                for (i,rect) in enumerate(faces):
                    shape = shape_predictor(gray_image,rect)
                    shape = face_utils.shape_to_np(shape)

                    for (name, (i,j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
                        if(name=='mouth'):
                            #cpyImage = image.copy()
                            X_frame = []
                            Y_frame = []
                            for (x,y) in   shape[i:j]:
                                X_frame.append(x)
                                Y_frame.append(y)
                    X.append(X_frame)
                    Y.append(Y_frame)

        except Exception as e:
            print("ERROR OCCURED......." + e)
            break;
    cap.release()
    cv2.destroyAllWindows()
    return X,Y

def process_Arrays(X,Y):
    X_sd_array = []
    Y_sd_array = []
    X_sd_array = np.std(X,axis = 0).astype('float32')
    Y_sd_array = np.std(Y,axis = 0).astype('float32')
    return X_sd_array,Y_sd_array


def process(training_Dataframe,video):

    X,Y = process_Video(video)
    X,Y = process_Arrays(X,Y)
    feature_vector = np.r_[X,Y]
    return feature_vector




In [38]:
filetotest = "/media/vinay/Academy/Final_year_project/Rohith_mp4_edited_in_windows/1_sec_Rohith_Avanu.mp4"
training_Dataframe = pd.DataFrame(dtype='float32')
training_Dataframe = process(training_Dataframe,filetotest)
print(training_Dataframe)

FPS : 30.0
 video/camera opened...
********************************************
Frame ID :0.0
Frame ID :1.0
Frame ID :2.0
Frame ID :3.0
Frame ID :4.0
Frame ID :5.0
Frame ID :6.0
Frame ID :7.0
Frame ID :8.0
Frame ID :9.0
Frame ID :10.0
Frame ID :11.0
Frame ID :12.0
Frame ID :13.0
Frame ID :14.0
Frame ID :15.0
Frame ID :16.0
Frame ID :17.0
Frame ID :18.0
Frame ID :19.0
Frame ID :20.0
Frame ID :21.0
Frame ID :22.0
Frame ID :23.0
Frame ID :24.0
Frame ID :25.0
Frame ID :26.0
Frame ID :27.0
Frame ID :28.0
Frame ID :29.0
[3.9255574 3.7452192 3.2138588 3.3099177 2.9066973 2.8223317 3.3975482
 3.0559595 3.234536  3.6763508 3.7806525 4.1821046 4.071036  3.2491024
 3.2653058 2.9066973 3.431229  3.0955164 3.74774   3.7835903 2.752978
 2.1052845 2.5166116 2.4677925 2.5897233 2.196209  2.7890658 3.6968455
 4.261064  4.3010335 4.3645034 3.81532   2.6674998 2.0912516 1.9507834
 2.0548046 2.6373386 4.6342926 4.7168727 4.8078637]


In [39]:
X_Predict = np.array(training_Dataframe)
X_Predict = X_Predict.reshape(1*8,5)
dat_fi1=np.array([])
for a in range(8):
    for b in range(5):
        if b==0:
            first1 =  X_Predict[a][b]
        else:
            second1 = X_Predict[a][b]
            ec1 = first1 - second1
            dat_fi1 = np.append(dat_fi1,ec1,axis=None)
            first1 =second1

In [40]:
X_Predict = dat_fi1.reshape(1,8,4)

In [41]:
AUDIO_FILE_NAME='test1.wav'

from moviepy import editor

def convert_vedio_to_audio(video):
    try:
        os.system('ffmpeg -i {} -acodec pcm_s16le -ar 16000 {}'.format(filetotest, AUDIO_FILE_NAME))
        return True,AUDIO_FILE_NAME
    except Exception as e:
       print(e)
       print("conversion from video to audio failed ")
    return False,AUDIO_FILE_NAME
    

def extract_feature(audio_file_name):
    X, sample_rate = librosa.load(audio_file_name,sr = 16000)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    return mfccs,chroma,mel,contrast,tonnetz
 

def process_audio_features(mfccs,chroma,mel,contrast,tonnetz):   

        features_array = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        return features_array
    

def get_audio_feature_vector(video_location):
    isConverted,audio_file_name = convert_vedio_to_audio(video_location)
    if(isConverted):
         mfccs,chroma,mel,contrast,tonnetz =  extract_feature(audio_file_name)
         feature_vector = process_audio_features(mfccs,chroma,mel,contrast,tonnetz)
         feature_vector  = np.around(feature_vector,decimals=3)
         return feature_vector
    else:
       print('Video conversion failed...')

In [42]:
X_Predict_Audio = get_audio_feature_vector(filetotest)
os.remove("test1.wav")

In [43]:
X_Predict_Audio.reshape(1,193,)

array([[-4.00861e+02,  1.17826e+02, -5.93000e+00, -7.08000e-01,
        -4.70000e-01, -5.45300e+00,  8.52300e+00, -1.29890e+01,
        -4.30400e+00, -1.19880e+01,  2.77400e+00, -4.69900e+00,
        -2.05700e+00, -2.49200e+00, -6.67100e+00, -2.64000e+00,
         7.61000e-01, -6.65000e-01,  1.39900e+00, -3.11000e+00,
         4.28000e-01,  3.84200e+00, -5.34100e+00,  1.57800e+00,
        -5.24400e+00,  4.09000e-01, -2.95900e+00,  2.14200e+00,
        -5.00700e+00, -3.09000e+00, -5.75800e+00, -3.13200e+00,
        -6.42300e+00, -9.57000e-01, -3.49800e+00, -2.92700e+00,
        -2.80600e+00, -1.33400e+00, -3.04900e+00, -1.78600e+00,
         7.80000e-01,  7.18000e-01,  8.32000e-01,  6.98000e-01,
         7.56000e-01,  6.70000e-01,  6.53000e-01,  6.47000e-01,
         5.97000e-01,  7.05000e-01,  7.16000e-01,  6.85000e-01,
         1.00000e-03,  1.10000e-02,  1.20000e-02,  3.50000e-02,
         4.13000e-01,  3.76000e-01,  6.00000e-03,  1.20000e-02,
         4.40000e-02,  1.48000e-01,  5.5

In [44]:
predict_prob=model.predict([X_Predict_Audio.reshape(1,193,),X_Predict.reshape(1,8,4)])
index=np.argmax(predict_prob[0])
#print(index)
print(words_list[index])

Avanu
